# Defining GAN Model

<table>
    <tr>
        <th><img src="./images/GeneralFramework.png" align-items="left"></th>
    </tr>
</table>

In [ ]:
from keras.optimizers import Adam
from keras.layers import Input
from keras.layers import Conv2D, Conv2DTranspose, Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.core import Activation
from keras.models import Model
from keras.layers.core import Flatten
from keras.layers import Dense

In [ ]:

def GAN_Model(noise_shape, image_shape, loss, optimizer_type, metrics):
    
    # Define noise vector
    Z = Input(shape = noise_shape)

    # Get generator model
    G = Generator(noise_shape)

    # Get discriminator model
    D = Discriminator(image_shape)

    # GAN Model
    D.trainable = False
    GAN = Model(input = Z, output = D(G(Z)))
    GAN.compile(loss = loss, optimizer = optimizer_type, metrics = metrics)
    GAN.summary()
    
    return GAN, G, D

# Generator $G$ Model


## Kernel initialization notes

<code>kernel_initializer = 'glorot_uniform'</code> _(being used)_ 

- The Glorot uniform initializer, also called Xavier uniform initializer.

- It draws samples from a uniform distribution within [-limit, limit] where limit is sqrt(6 / (fan_in + fan_out)) where fan_in is the number of input units in the weight tensor and fan_out is the number of output units in the weight tensor.

<code>kernel_initializer = RandomNormal(mean=0.0, stddev=0.01)</code>

- Outputs random values from a normal distribution.

## First Layer Definition Notes
### Padding:
- Changing padding = 'same' in the first layer makes a lot fo difference!!!!
- But he is using in the first layer <code>padding = "valid"</code>

## LeakReLU

The derivative of the LeayReLU is 1 in the positive part, and is a small fraction in the negative part.

Leaky version of a Rectified Linear Unit: It allows a small gradient when the unit is not active: 

$f(x) = alpha * x$, for $x < 0$

$f(x) = x$, for $x >= 0$



[Tips](https://www.quora.com/What-are-the-advantages-of-using-Leaky-Rectified-Linear-Units-Leaky-ReLU-over-normal-ReLU-in-deep-learning)

In [ ]:
def Generator(noise_shape):
    
    """
    Options
    """
    # Conv2D options
    df = "channels_last"  # data_format
    ki = 'glorot_uniform' # kernel_initializer
    
    # LeakyReLU option
    alpha = 0.2
    
    # BatchNormalization option
    momentum = 0.5
    
    # Loss Function 
    loss = 'binary_crossentropy'

    # Optimizer option
    gen_opt = Adam(lr = 0.00015, beta_1 = 0.5)
        
    # Metric type 
    metrics = ['accuracy']    
    
    
    """
    Input Layer {noise}
    """    
    gen_input = Input(shape = noise_shape)

    
    """
    Layer (1) {Conv2DTranspose - BatchNormalization - LeakyReLU}
    """
    f = 512     # amount of filters    
    k = (4,4)   # kernel_size
    s = (1,1)   # strides    
    p = "valid" # padding
    
    generator = Conv2DTranspose(filters = f, kernel_size = k, strides = s, padding = p, data_format = df, kernel_initializer = ki)(gen_input)
    generator = BatchNormalization(momentum = momentum)(generator)
    generator = LeakyReLU(alpha=alpha)(generator)
      
        
    """
    Layer (2) {Conv2DTranspose - BatchNormalization - LeakyReLU}
    """
    f = 256    # amount of filters    
    k = (4,4)  # kernel_size
    s = (2,2)  # strides    
    p = "same" # padding
    
    generator = Conv2DTranspose(filters = f, kernel_size = k, strides = s, padding = p, data_format = df, kernel_initializer = ki)(generator)
    generator = BatchNormalization(momentum = momentum)(generator)    
    generator = LeakyReLU(alpha=alpha)(generator)

    
    """
    Layer (3) {Conv2DTranspose - BatchNormalization - LeakyReLU}
    """
    f = 128 # amount of filters    
    
    generator = Conv2DTranspose(filters = f, kernel_size = k, strides = s, padding = p, data_format = df, kernel_initializer = ki)(generator)
    generator = BatchNormalization(momentum = momentum)(generator)    
    generator = LeakyReLU(alpha=alpha)(generator)    

    """
    Layer (4) {Conv2DTranspose - BatchNormalization - LeakyReLU}
    """
    f = 64 # amount of filters    
    
    generator = Conv2DTranspose(filters = f, kernel_size = k, strides = s, padding = p, data_format = df, kernel_initializer = ki)(generator)
    generator = BatchNormalization(momentum = momentum)(generator)    
    generator = LeakyReLU(alpha=alpha)(generator)    
    
    """
    Layer (5) {Conv2DTranspose - BatchNormalization - LeakyReLU}
    """    
    f = 32 # amount of filters

    generator = Conv2DTranspose(filters = f, kernel_size = k, strides = s, padding = p, data_format = df, kernel_initializer = ki)(generator)
    generator = BatchNormalization(momentum = momentum)(generator)    
    generator = LeakyReLU(alpha=alpha)(generator)    
    
    """
    Layer (6) {Conv2DTranspose - BatchNormalization - LeakyReLU}
    """
    f = 16 # amount of filters

    generator = Conv2DTranspose(filters = f, kernel_size = k, strides = s, padding = p, data_format = df, kernel_initializer = ki)(generator)
    generator = BatchNormalization(momentum = momentum)(generator)
    generator = LeakyReLU(alpha=alpha)(generator)
        
    """
    Layer (7) {Conv2D - BatchNormalization - LeakyReLU}
    """    
    f = 16    # amount of filters
    k = (3,3) # kernel_size
    s = (1,1) # strides

    generator = Conv2D(filters = f, kernel_size = k, strides = s, padding = p, data_format = df, kernel_initializer = ki)(generator)
    generator = BatchNormalization(momentum = momentum)(generator)
    generator = LeakyReLU(alpha=alpha)(generator)
     
    """
    Layer (8) {Conv2DTranspose - Activation}
    """
    f = 3     # amount of filters
    k = (4,4) # kernel_size
    s = (2,2) # strides
    
    generator = Conv2DTranspose(filters = f, kernel_size = k, strides = s, padding = p, data_format = df, kernel_initializer = ki)(generator)
    generator = Activation('tanh')(generator)
        
    """
    Model
    """   
    generator_model = Model(input = gen_input, output = generator)
    generator_model.compile(loss = loss, optimizer = gen_opt, metrics = metrics)
    generator_model.summary()

    return generator_model


# Discriminator $D$ Model

In [ ]:
def Discriminator(image_shape):   
    
    """
    Options
    """
    # Conv2D options
    k = (4,4)             # kernel_size
    s = (2,2)             # strides
    p = "same"            # padding
    df = "channels_last"  # data_format
    ki = 'glorot_uniform' # kernel_initializer
    
    # LeakyReLU option
    alpha = 0.2
    
    # BatchNormalization option
    momentum = 0.5
    
    # Loss Function 
    loss = 'binary_crossentropy'

    # Optimizer option
    dis_opt = Adam(lr = 0.0002, beta_1 = 0.5)
    
    # Metric type 
    metrics = ['accuracy']    
    
    
    """
    Input Layer 
    """
    dis_input = Input(shape = image_shape)
        
    """
    Layer (1) {Conv2D - LeakyReLU}
    """
    f = 64 # amount of filters
    discriminator = Conv2D(filters = f, kernel_size = k , strides = s, padding = p, data_format = df, kernel_initializer = ki)(dis_input)
    discriminator = LeakyReLU(alpha = alpha)(discriminator)
    
    """
    Layer (2) {Conv2D - BatchNormalization- LeakyReLU}
    """    
    f = 128 # amount of filters
    discriminator = Conv2D(filters = f, kernel_size = k , strides = s, padding = p, data_format = df, kernel_initializer = ki)(discriminator)
    discriminator = BatchNormalization(momentum = momentum)(discriminator)
    discriminator = LeakyReLU(alpha = alpha)(discriminator)

    """
    Layer (3) {Conv2D - BatchNormalization- LeakyReLU}
    """
    f = 256 # amount of filters
    discriminator = Conv2D(filters = f, kernel_size = k , strides = s, padding = p, data_format = df, kernel_initializer = ki)(discriminator)
    discriminator = BatchNormalization(momentum = momentum)(discriminator)
    discriminator = LeakyReLU(alpha = alpha)(discriminator)
    
    """
    Layer (4) {Conv2D - BatchNormalization- LeakyReLU}
    """
    f = 512 # amount of filters
    discriminator = Conv2D(filters = f, kernel_size = k , strides = s, padding = p, data_format = df, kernel_initializer = ki)(discriminator)
    discriminator = BatchNormalization(momentum = momentum)(discriminator)
    discriminator = LeakyReLU(alpha = alpha)(discriminator)

    """
    Layer (5) {Flatten}
    """        
    discriminator = Flatten()(discriminator)
    
    """
    Layer (6) {Dense - Activation}
    """  
    discriminator = Dense(1)(discriminator)
    discriminator = Activation('sigmoid')(discriminator)
    
    """
    Model
    """      
    discriminator_model = Model(input = dis_input, output = discriminator)
    discriminator_model.compile(loss = loss, optimizer = dis_opt, metrics = metrics)
    discriminator_model.summary()
    
    return discriminator_model
